# Emotion Conditioned Music Generation
This notebook provides the code for implementing a Transformer-GAN for the dissertation. The objective of the model is to produce sentimental music given an input emotion


## Importing libraries

In [1]:
# !pip install music21 miditoolkit miditok

In [2]:
# !pip uninstall torch
# !pip install --user torch==1.7.0 torchvision==0.8.1 -f https://download.pytorch.org/whl/cu102/torch_stable.html

In [3]:
# !pip install numpy pandas tensorflow sklearn

In [1]:
import numpy as np 
import pandas as pd 
from io import open
import tensorflow as tf
import glob


import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import time
from miditok import get_midi_programs, REMI, MIDILike
from miditoolkit import MidiFile
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split

C:\ProgramData\Anaconda3\envs\newpydisser\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.__version__

'1.7.0'

In [3]:
device = 'cuda'

In [4]:
torch.cuda.empty_cache()

In [5]:
torch.cuda.is_available()

True

In [6]:
# Seed
# seed = 22
# torch.manual_seed(seed)
# torch.cuda.manual_seed(seed)
# torch.cuda.manual_seed_all(seed)
# np.random.seed(seed)
# torch.backends.cudnn.benchmark = False
# torch.backends.cudnn.deterministic = True

## Loading the Dataset

In [7]:
# how a midi file looks like
midi = MidiFile('archive/EMOPIA_1.0 (1)/EMOPIA_1.0/midis/Q1__8v0MFBZoco_0.mid')
midi

ticks per beat: 384
max tick: 46051
tempo changes: 1
time sig: 1
key sig: 0
markers: 0
lyrics: False
instruments: 1

In [8]:
# for now, we will only be using for piano right since it determines the melody
midi.instruments

[Instrument(program=0, is_drum=False, name="")]

In [9]:
# file path to the MIDI files
files_paths = list(glob.glob('archive/EMOPIA_1.0 (1)/EMOPIA_1.0/midis/*.mid'))
# reading labels
labels_df = pd.read_csv('archive/EMOPIA_1.0 (1)/EMOPIA_1.0/label.csv')
labels_df = list(labels_df['4Q'])

In [10]:
import muspy

def return_range(music):
    h = 0
    l = 127
    for track in music.tracks:
        for note in track.notes:
            if note.pitch > h:
                h = note.pitch
            if note.pitch < l:
                l = note.pitch
    return [h, l]


tempos = []
pitches = []

for file in files_paths:
    music = muspy.read_midi(file)
    tempos.append(music.tempos[0].qpm)
    pitches.extend(return_range(music))

In [11]:
print("The unique tempos found in the dataset are: ", set(tempos))
print('minimum pitch found', min(pitches))
print('maximum pitch found', max(pitches))

The unique tempos found in the dataset are:  {120.0}
minimum pitch found 22
maximum pitch found 105


In [12]:
pitch_range = range(22, 105)
additional_tokens = {'Chord': True, 'Rest': True, 'Tempo': True, 'Program': False,
                     'rest_range': (2, 4),  # (half, 8 beats)
                     'nb_tempos': 32,  # nb of tempo bins
                     'tempo_range': (100, 140),
                     'TimeSignature':None}  # (min, max)

In [13]:
# create a list of notes
# this stores the REMI encoded tokens of the midi files

def load_files(files_paths, encoder = REMI()):
    assert len(files_paths) > 0
    notes = []

    for file in files_paths:
        # file_name = os.path.basename(file)

        # read the MIDI file
        midi = MidiFile(file)

        # Converts MIDI to tokens
        tokens = encoder.midi_to_tokens(midi)
        
        # The EMOPIA dataset has midi files with only one instrument, i.e. the piano 
        # hence we just add those tokens
        notes.append(tokens[0])

    return notes, encoder

In [14]:
notes, midi_enc = load_files(files_paths, MIDILike(pitch_range, additional_tokens=additional_tokens))

In [15]:
print("There are",len(midi_enc.vocab),"unique tokens in the files")

There are 317 unique tokens in the files


In [52]:
# Create a dataset corpus from the notes and labels
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler

class Corpus(Dataset):
    def __init__(self, notes, labels, encoder, seq_length):
        self.encoder = encoder
        self.seq_len = seq_length

       
        self.xtrain, self.ytrain= self.tokenize(notes, labels)
        # self.xtest, self.ytest, _, _ = self.tokenize(ntest, ltest)
        # self.xvalid = self.tokenize(ntest, ltest)
   
    def __len__(self):
        return len(self.encoder.vocab)

    def len_dataset(self):
        return len(self.xtrain)
   
    def __getitem__(self, index, ):
        return self.xtrain[index], self.ytrain[index]
   
    def tokenize(self, notes, labels):
        assert len(notes) > 0
        assert len(labels) > 0

        # create a set of notes
        # they should all be padded to have sequence of len seq_len
        songss = []
        labelss = []

        for song, label in zip(notes, labels):
            song = torch.tensor(song).type(torch.int64)
            songs = list(song.split(self.seq_len))

            for i in range(len(songs)):
                # removing sequences that have < seq len/4 tokens
                if len(songs[i]) < self.seq_len/4:
                    del songs[i]
                    continue
                labelss.append(label-1)
            songss.extend(songs)
       
        # padding songs to be of same length
        songs = pad_sequence(songss)

        corpus = []

        # adding emotion values to the sequences
        for song, label in zip(songs.view(songs.size(1), songs.size(0)), labelss):
            l = torch.full((self.seq_len,1), label)
            song = song.view(song.size(0), 1)
            inp = torch.cat([song, l], dim=-1)
            corpus.append(inp)

        corpus = torch.stack(corpus)


        data = corpus[:,:self.seq_len - 1, :]
        target = corpus[:,1:self.seq_len, :]

        return data, target

In [53]:
ntrain, ntest, ltrain, ltest = train_test_split(notes, labels_df, test_size=0.3, random_state=42, shuffle=True, stratify=labels_df)
train_corpus = Corpus(ntrain, ltrain, midi_enc, 21)
val_corpus = Corpus(ntest, ltest, midi_enc, 21)

In [54]:
xtest = val_corpus.xtrain
xtrain = train_corpus.xtrain


print("train data shape:", train_corpus.xtrain.shape)
print("test data shape:", val_corpus.xtrain.shape)
print("train data shape:", train_corpus.ytrain.shape)
print("test data shape:", val_corpus.ytrain.shape)

train data shape: torch.Size([35691, 20, 2])
test data shape: torch.Size([15507, 20, 2])
train data shape: torch.Size([35691, 20, 2])
test data shape: torch.Size([15507, 20, 2])


In [55]:
batch_size = 32
# creating a dataloader
train_dataloader = DataLoader(
    train_corpus,
    sampler=SequentialSampler(xtrain),
    batch_size=batch_size,
)
val_dataloader = DataLoader(
    val_corpus,
    sampler=SequentialSampler(xtest),
    batch_size=batch_size,
)

In [56]:
print("There are total",len(notes), "songs and a total of", len(xtest) + len(xtrain), "sequences extracted")

There are total 1078 songs and a total of 51198 sequences extracted


## Model Building

### Constants

In [57]:
# NEW for every type of token: corpus and emotion
ntokens = [len(train_corpus), 4]

emsize = 256
nhead = 4
nhid = 128
nlayer = 2
dropout = 0.2
# Loop over epochs.
lr = 0.001
best_val_loss = None
epochs = 120
save = './model.pt'
criterion = nn.CrossEntropyLoss()
device = device


### Position Encoding

In [58]:
# adapted from the pytorch positional encoding class
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=100):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # PE is the Positional Encoding matrix 
        # THIS STORES THE POSITIONS OF THE SEQUENCE
        pe = torch.zeros(max_len, d_model)

        # Arange - RETURNS A RANGE BETWEEN VALUES, HERE IT IS 0 - max_len
        # unsqueeze - adds a dimension, 1 means that each element in the first list is now in a list
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        # division term, here it is (10000 ** ((2 * i)/d_model))
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

        # calculating the position encoding for the even and odd terms        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        # Unsqueeze 0 will put PE in one list
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        # make embeddings relatively larger
        # This is so we do not lose the importance of the embedding
        # we add the embedding to the PE 
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [59]:
class MIDITransformer(nn.Module):
    """Container module with an encoder, a recurrent or transformer module, and a decoder."""

    def __init__(self, ntoken, d_model, nhead, nlayers, dropout=0.5, max_length = 100, device = device):
        super(MIDITransformer, self).__init__()
        try:
            from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoder, TransformerDecoderLayer
        except:
            raise ImportError('TransformerEncoder module does not exist in PyTorch 1.1 or lower.')

        # original mask
        self.src_mask = None
        self.max_length = max_length
        self.d_model = d_model
        self.nlayers = nlayers
        self.ntokens = ntoken

        self.device = device

        # NEW criterion and embedding size
        self.criterion = nn.CrossEntropyLoss(reduction='none')
        # CHANGED: using embedding size and reshaping vector
        self.embed_siz = [128, 128]

        # embedding encoding
        self.embedding_notes  = nn.Embedding(self.ntokens[0], self.embed_siz[0])
        self.embedding_emotion   = nn.Embedding(self.ntokens[1], self.embed_siz[1])
        
        self.in_linear = nn.Linear(np.sum(self.embed_siz), d_model)
        self.target_linear = nn.Linear(self.embed_siz[0], d_model)
        # positional encoding
        self.pos_encoder = PositionalEncoding(d_model, dropout, max_length)

        # in linear layer
        # CHANGED: using this to convert one hot encoding of emotions batch * 5 -> linear transformation of emotions batch * 
        # TODO
        self.linear = nn.Linear(np.sum(self.embed_siz), self.d_model)
        
        # encoder
        encoder_layer = TransformerEncoderLayer(d_model = d_model, nhead = nhead, dropout = dropout)
        self.encoder = TransformerEncoder(encoder_layer, nlayers)
        
        # decoder
        decoder_layer = TransformerDecoderLayer(d_model = d_model, nhead = nhead, dropout = dropout)
        self.decoder = TransformerDecoder(decoder_layer, nlayers)

        # output layers
        self.project_notes = nn.Linear(d_model, ntoken[0])
        # self.project_emo = nn.Linear(d_model, ntoken[1])
        
        
        self.init_weights()
    
    def compute_loss(self, predict, target):
        loss = self.criterion(predict, target)
        return torch.sum(loss)
            

    def _generate_square_subsequent_mask(self, sz):
        mask = torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)
        return mask

    def init_weights(self):
        initrange = 0.1
        nn.init.uniform_(self.embedding_notes.weight, -initrange, initrange)
        nn.init.uniform_(self.embedding_emotion.weight, -initrange, initrange)
     
        self.linear.bias.data.zero_()
        self.linear.weight.data.uniform_(-initrange, initrange)
        self.project_notes.bias.data.zero_()
        self.project_notes.weight.data.uniform_(-initrange, initrange)
        # self.project_emo.bias.data.zero_()
        # self.project_emo.weight.data.uniform_(-initrange, initrange)

    def forward(self, x_note, x_emo, y_note, src_mask = None):
        # creating embedding for the notes and emotions
        x_note = self.embedding_notes(x_note)
        x_emo = self.embedding_emotion(x_emo)

        # normalising the input for the position encoding
        x_note = x_note * math.sqrt(self.d_model)
        x_emo = x_emo * math.sqrt(self.d_model)

        # concatenating as one input
        x = torch.cat([x_note, x_emo], dim=-1)

        # sending through linear layer
        x = self.in_linear(x)

        x = self.pos_encoder(x)

        if src_mask == None:
            src_mask = self._generate_square_subsequent_mask(x.size(1)).to(self.device)
            
        self.src_mask = src_mask

        output = self.encoder(x.view(x.size(1), x.size(0), x.size(2)), self.src_mask)
        
        # creating embedding for the notes and emotions
        y_note = self.embedding_notes(y_note)

        # normalising the input for the position encoding
        y_note = y_note * math.sqrt(self.d_model)
        
        # sending through linear layer
        y_note = self.target_linear(y_note)

        y_note = self.pos_encoder(y_note)
        
        output = self.decoder(y_note.view(y_note.size(1), y_note.size(0), y_note.size(2)), output)

        return self.project_notes(output)

In [60]:
# get_batch subdivides the source data into chunks of length args.bptt.
# If source is equal to the example output of the batchify function, with
# a bptt-limit of 2, we'd get the following two Variables for i = 0:
# ┌ a g m s ┐ ┌ b h n t ┐
# └ b h n t ┘ └ c i o u ┘
# Note that despite the name of the function, the subdivison of data is not
# done along the batch dimension (i.e. dimension 1), since that was handled
# by the batchify function. The chunks are along dimension 0, corresponding
# to the seq_len dimension in the LSTM.
def get_batch(source, batch_size):
    rand_columns = torch.randperm(source.size(0))[:batch_size]
    # batch_size = min(batch_size, len(source) - 1 - i)
    data = source[rand_columns,:source.size(1)-1, :]
    target = source[rand_columns,1:source.size(1), :]
    return data, target

In [61]:
def generate_square_subsequent_mask(sz):
    """Generates an upper-triangular matrix of -inf, with zeros on diag."""
    return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)

In [62]:
model = MIDITransformer(ntokens, emsize, nhead, nlayer, dropout, device=device)
model.to(device)

MIDITransformer(
  (criterion): CrossEntropyLoss()
  (embedding_notes): Embedding(317, 128)
  (embedding_emotion): Embedding(4, 128)
  (in_linear): Linear(in_features=256, out_features=256, bias=True)
  (target_linear): Linear(in_features=128, out_features=256, bias=True)
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (linear): Linear(in_features=256, out_features=256, bias=True)
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): _LinearWithBias(in_features=256, out_features=256, bias=True)
        )
        (linear1): Linear(in_features=256, out_features=2048, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
        (linear2): Linear(in_features=2048, out_features=256, bias=True)
        (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (

In [63]:
def network_paras(model):
    # compute only trainable params
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    params = sum([np.prod(p.size()) for p in model_parameters])
    return params

print("There are",network_paras(model),"parameters in the model")

There are 6074813 parameters in the model


In [64]:
optim = torch.optim.Adam(model.parameters(), lr = lr)
scheduler = torch.optim.lr_scheduler.StepLR(optim, 1.0, gamma = 0.95)

In [65]:
import torch
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

## Training

In [66]:
def train(model, epochs = 10):

    model.train()  # turn on train mode
    
    for epoch in range(epochs):
        total_loss = 0.
        log_interval = 1
        start_time = time.time()
        per_batch = 20
        # src_mask = generate_square_subsequent_mask(2047).to(device)
        

        num_batches = len(train_dataloader)

    
        for bidx, (data, targets) in enumerate(train_dataloader):
            optim.zero_grad()

            src_mask = generate_square_subsequent_mask(data.size(1)).to(device)

            output = model(data[:,:,0].to(device), data[:,:,1].to(device), targets[:,:,0].to(device), src_mask.to(device))

            loss = criterion(output.view(output.size(1), output.size(2), output.size(0)).cpu(), targets[:,:,0].cpu())
            
                
            writer.add_scalar("Loss/output/train", loss, epoch)

            loss.backward(retain_graph = True)

            optim.step()

            
            total_loss += loss

            if epoch % log_interval == 0 and epoch > 0:

                lr = scheduler.get_last_lr()[0]
                ms_per_batch = (time.time() - start_time) * 1000 / log_interval
                cur_loss = total_loss / log_interval
                ppl = math.exp(cur_loss)

                print(f'| epoch {epoch:3d} | '
                    f'learning rate {lr:02.4f} | {ms_per_batch:5.2f} ms | '
                    f'loss {cur_loss:5.2f}')

                total_loss = 0
                start_time = time.time()
 

In [67]:
train(model)

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 8.00 GiB total capacity; 6.65 GiB already allocated; 0 bytes free; 6.76 GiB reserved in total by PyTorch)

In [ ]:
writer.close()

In [ ]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

Model's state_dict:
embedding_notes.weight 	 torch.Size([273, 128])
embedding_emotion.weight 	 torch.Size([4, 128])
in_linear.weight 	 torch.Size([512, 256])
in_linear.bias 	 torch.Size([512])
pos_encoder.pe 	 torch.Size([5000, 1, 512])
linear.weight 	 torch.Size([512, 256])
linear.bias 	 torch.Size([512])
encoder.layers.0.self_attn.in_proj_weight 	 torch.Size([1536, 512])
encoder.layers.0.self_attn.in_proj_bias 	 torch.Size([1536])
encoder.layers.0.self_attn.out_proj.weight 	 torch.Size([512, 512])
encoder.layers.0.self_attn.out_proj.bias 	 torch.Size([512])
encoder.layers.0.linear1.weight 	 torch.Size([2048, 512])
encoder.layers.0.linear1.bias 	 torch.Size([2048])
encoder.layers.0.linear2.weight 	 torch.Size([512, 2048])
encoder.layers.0.linear2.bias 	 torch.Size([512])
encoder.layers.0.norm1.weight 	 torch.Size([512])
encoder.layers.0.norm1.bias 	 torch.Size([512])
encoder.layers.0.norm2.weight 	 torch.Size([512])
encoder.layers.0.norm2.bias 	 torch.Size([512])
encoder.layers.1.self

In [ ]:
torch.save(model.state_dict(), './models/midi_transformer.pt')

## Generate

In [ ]:
model = MIDITransformer(ntokens, emsize, nhead, nlayer, dropout, device=device)
model.load_state_dict(torch.load('./models/midi_transformer.pt'))
model.to(device)
model.eval()

MIDITransformer(
  (criterion): CrossEntropyLoss()
  (embedding_notes): Embedding(273, 128)
  (embedding_emotion): Embedding(4, 128)
  (in_linear): Linear(in_features=256, out_features=512, bias=True)
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (linear): Linear(in_features=256, out_features=512, bias=True)
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): _LinearWithBias(in_features=512, out_features=512, bias=True)
        )
        (linear1): Linear(in_features=512, out_features=2048, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
        (linear2): Linear(in_features=2048, out_features=512, bias=True)
        (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.2, inplace=False)
        (dropout2): Dropout(p=0

In [ ]:
# tensorboard
# https://pytorch.org/tutorials/recipes/recipes/tensorboard_with_pytorch.html?msclkid=ce0b97e5b41911ec9d2e71bb3c7d0f90

In [ ]:
# !pip install muspy
import muspy

In [ ]:
len(corpus)

273

In [ ]:
sequences = []
for k in range(3):
    print(i)
    for emo in range(4):
        n_generate = 4000
        temperature = 1
        sequence = []
        log_interval = 4000 # interval between logs
        input = torch.randint(len(corpus), (1, 2), dtype=torch.long).to(device)
        emotion = torch.full((1, 2), emo).to(device)
        # emotion[:,0] = emo


        src_mask = generate_square_subsequent_mask(len(input)).to(device)
        with torch.no_grad():  # no tracking history
            for i in range(n_generate):
                output, _ = model(input, emotion)
                F.log_softmax(output, dim=-1)

                word_weights = output[-1].squeeze().div(temperature).exp().cpu()
                word = torch.multinomial(word_weights, 1)[0].tolist()
                word_tensor = torch.Tensor([[word]]).long().to(device)
                # print(input.shape)
                # print(word_tensor.shape)
                input = torch.cat([input, word_tensor], 1)
                emotion = torch.cat([emotion, torch.zeros((1,1), dtype=int).to(device)], -1)

                
                sequence.append(word)

            if i % log_interval == 0:
                print('| Generated {}/{} notes'.format(i, n_generate))
        sequences.append(sequence)

0
3999
3999


In [ ]:
len(sequences[0])

4000

In [ ]:
import muspy

In [ ]:
q1 = [sequences[0], sequences[4], sequences[8]]
q2 = [sequences[1], sequences[5], sequences[9]]
q3 = [sequences[2], sequences[6], sequences[10]]
q4 = [sequences[3], sequences[7], sequences[11]]

In [ ]:
date = '13_04_'
pitch_ranges = []
n_pitches = []
polyphonies = []
empty_beat_rates = []

In [ ]:
for i,seq in enumerate(q4):
    # TODO: remove this
    # seq = seq[0]

    converted_back_midi = midi_enc.tokens_to_midi([seq], get_midi_programs(midi))
    file_name = 'midi_transformer_' + date + str(i) + '_' + str(4) + '.mid'
    converted_back_midi.dump(file_name)
    music = muspy.read_midi(file_name)
    pitch_range = muspy.pitch_range(music)
    n_pitches_used = muspy.n_pitches_used(music)
    polyphony = muspy.polyphony(music) # average number of pitches being played concurrently.
    empty_beat_rate = muspy.empty_beat_rate(music)

    # music = muspy.read_midi(file_name)
    pitch_ranges.append(muspy.pitch_range(music))
    n_pitches.append(muspy.n_pitches_used(music))
    polyphonies.append(muspy.polyphony(music)) # average number of pitches being played concurrently.
    empty_beat_rates.append(muspy.empty_beat_rate(music))

midi_transformer_13_04_0_4.mid
midi_transformer_13_04_1_4.mid
midi_transformer_13_04_2_4.mid


In [ ]:
results_transgan = {'Pitch_range': pitch_ranges, 'Num_pitches': n_pitches, 'Polyphony': polyphonies, 'Empty_beat_rates': empty_beat_rates}
results_df = pd.DataFrame(results_transgan)
results_df.to_csv('midi_transformer.csv')

In [ ]:
converted_back_midi

ticks per beat: 384
max tick: 0
tempo changes: 1
time sig: 0
key sig: 0
markers: 0
lyrics: False
instruments: 1

## Metrics

### BLEU Score

In [ ]:
train_check = train_data[:,:,0]
train_check.shape

torch.Size([50077, 21])

In [ ]:
gen_check = []
for sequence in sequences:
    # print(sequence[0])
    for i in range(0, len(sequence)-21, 21):
        gen_check.append(sequence[i:i+21])

In [ ]:
torch.Tensor(gen_check).shape

torch.Size([2280, 21])

In [ ]:
# !pip install nltk

In [ ]:
from nltk.translate.bleu_score import corpus_bleu

score = corpus_bleu([train_check], [torch.Tensor(gen_check)])
score

### MusPy metrics

In [ ]:
results_df.describe()

,Pitch_range,Num_pitches,Polyphony,Empty_beat_rates
count,12.000000,12.000000,12.000000,12.000000
mean,60.750000,5.083333,1.002694,0.989892
std,17.664165,2.234373,0.009332,0.004871
min,28.000000,2.000000,1.000000,0.979579
25%,51.500000,3.750000,1.000000,0.987825
50%,62.000000,4.500000,1.000000,0.990643
75%,73.250000,6.500000,1.000000,0.992974
max,85.000000,9.000000,1.032328,0.996146
